In [2]:
pip install boto3

     |████████████████████████████████| 139 kB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 2.3 MB/s  eta 0:00:01
     |████████████████████████████████| 13.5 MB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 144 kB 6.8 MB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
Note: you may need to restart the kernel to use updated packages.


In [22]:
#new code
# ============================================================================
# IMPORTS
# ============================================================================
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import (
    col, max, lit, current_timestamp, current_date, explode_outer
)
from pyspark.sql.types import NullType, StringType, DoubleType, StructType, ArrayType,LongType
from datetime import datetime, timedelta
import os, re, json, boto3
from typing import Dict
from datetime import datetime, date

# ============================================================================
# CONFIG
# ============================================================================
RAW_PATH = "s3a://mailshake-analytics/raw"
CURATED_PATH = "s3a://mailshake-analytics/curated"
curated_base_path = "s3a://mailshake-analytics/curated"
raw_base_path = "s3a://mailshake-analytics/raw"
BUCKET = "mailshake-analytics"
CLIENTS_KEY = "config/clients_test.json"
RUN_DATE = datetime.utcnow().strftime("%Y-%m-%d")
SINGLE_DATE = None        # None for incremental activities
# ============================================================================
# SPARK SESSION
# ============================================================================
spark = (
    SparkSession.builder
    .appName("MailshakeCampaignCurations")
    .config(
        "spark.driver.extraClassPath",
        "/opt/spark/jars/hadoop-aws-3.3.4.jar:/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar"
    )
    .config(
        "spark.executor.extraClassPath",
        "/opt/spark/jars/hadoop-aws-3.3.4.jar:/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar"
    )
    .getOrCreate()
)
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

# ============================================================================
# S3 CONFIG
# ============================================================================
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.access.key", os.getenv("AWS_ACCESS_KEY_ID"))
hadoop_conf.set("fs.s3a.secret.key", os.getenv("AWS_SECRET_ACCESS_KEY"))
hadoop_conf.set("fs.s3a.endpoint", "s3.amazonaws.com")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

# ============================================================================
# CLIENT LOADING
# ============================================================================
s3 = boto3.client("s3")

def load_clients() -> Dict[str, Dict[str, str]]:
    obj = s3.get_object(Bucket=BUCKET, Key=CLIENTS_KEY)
    return json.loads(obj["Body"].read().decode("utf-8")).get("clients", {})

clients_dict = load_clients()
CLIENT_IDS = list(clients_dict.keys())
print(f"Loaded clients: {CLIENT_IDS}")

# ============================================================================
# HELPERS
# ============================================================================
def sanitize_column_names(df):
    for col_name in df.columns:
        clean = re.sub(r'[^a-zA-Z0-9_]', '_', col_name)
        clean = re.sub(r'_+', '_', clean).lower()
        if clean != col_name:
            df = df.withColumnRenamed(col_name, clean)
    return df

def fix_void_columns(df):
    for field in df.schema.fields:
        if isinstance(field.dataType, NullType):
            df = df.withColumn(field.name, col(field.name).cast(StringType()))
    return df

def ensure_columns_and_reorder(df: DataFrame, column_order: list, column_types: dict = None) -> DataFrame:
    column_types = column_types or {}
    for col_name in column_order:
        if col_name not in df.columns:
            dtype = column_types.get(col_name)
            if dtype:
                df = df.withColumn(col_name, lit(None).cast(dtype))
            else:
                df = df.withColumn(col_name, lit(None))
            print(f"⚠️ Adding missing column: {col_name}")
    return df.select([col(c) for c in column_order])

def flatten_struct_columns(df):
    while True:
        struct_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, StructType)]
        if not struct_cols: break
        for col_name in struct_cols:
            for nested in df.schema[col_name].dataType.fields:
                df = df.withColumn(f"{col_name}_{nested.name}", col(f"{col_name}.{nested.name}"))
            df = df.drop(col_name)

    array_struct_cols = [
        f.name for f in df.schema.fields
        if isinstance(f.dataType, ArrayType) and isinstance(f.dataType.elementType, StructType)
    ]
    for col_name in array_struct_cols:
        df = df.withColumn(col_name, explode_outer(col(col_name)))
        for nested in df.schema[col_name].dataType.fields:
            df = df.withColumn(f"{col_name}_{nested.name}", col(f"{col_name}.{nested.name}"))
        df = df.drop(col_name)
    return df
    



def get_dates_to_process(curated_path, raw_base_path, dataset_name, client_ids, single_date=None):
    """
    Returns dict:
    - client_id -> list of incremental event_dates that ACTUALLY exist in raw
    - Empty list => snapshot
    """
    s3_client = boto3.client("s3")
    bucket = "mailshake-analytics"

    # Manual override
    if single_date:
        return {c: [single_date] for c in client_ids}

    # Campaigns never use incremental
    if dataset_name.startswith("campaign"):
        return {c: [] for c in client_ids}

    # --- Read curated to get last processed date ---
    try:
        existing = spark.read.parquet(curated_path)
        last_dates = (
            existing.groupBy("client_id")
            .agg(max("source_date").alias("last_date"))
            .collect()
        )
        last_map = {r["client_id"]: r["last_date"] for r in last_dates}
        print("Loaded last_dates from curated:")
        for k, v in last_map.items():
            print(f"  {k}: {v}")
    except Exception:
        last_map = {}

    dates = {}

    for client in client_ids:
        # No curated → snapshot
        if client not in last_map:
            dates[client] = []
            continue

        last_date = datetime.strptime(str(last_map[client]), "%Y-%m-%d").date()

        # List S3 folders for this client & dataset
        prefix = f"raw/client_id={client}/entity={dataset_name}/"
        incremental_dates = []

        try:
            paginator = s3_client.get_paginator("list_objects_v2")
            pages = paginator.paginate(Bucket=bucket, Prefix=prefix, Delimiter='/')

            for page in pages:
                for cp in page.get("CommonPrefixes", []):
                    folder_name = cp.get("Prefix").rstrip('/').split('/')[-1]  # e.g., event_date=2026-01-03
                    if folder_name.startswith("event_date="):
                        d_str = folder_name.split("=")[1]
                        d_dt = datetime.strptime(d_str, "%Y-%m-%d").date()
                        print(f"Found S3 folder: {d_dt}")
                        if d_dt > last_date:
                            incremental_dates.append(d_str)

        except Exception as e:
            print(f"⚠️ Could not list raw path {prefix}: {e}")
            incremental_dates = []

        dates[client] = sorted(incremental_dates)
        print(f"{client} last_date={last_date}, incremental_dates={dates[client]}")

    return dates


def process_dataset(
    raw_base_path, curated_base_path, client_ids, dataset_name,
    unique_keys, explode_col=None, dates_per_client=None,
    desired_columns=None, column_types=None
):
    entity_path = f"{curated_base_path}/entity={dataset_name}"

    for client_id in client_ids:

        # ------------------------------------------------------------------
        # Decide snapshot vs incremental (ONCE)
        # ------------------------------------------------------------------
        curated_client_path = f"{curated_base_path}/entity={dataset_name}/client_id={client_id}"

        if dataset_name.startswith("campaign"):
            snapshot_mode = True
        else:
            try:
                spark.read.parquet(curated_client_path)
                snapshot_mode = False   # curated exists → incremental
            except Exception:
                snapshot_mode = True    # first run → snapshot

        # ------------------------------------------------------------------
        # Determine paths to process
        # ------------------------------------------------------------------
        paths_to_process = []

        if snapshot_mode:
            paths_to_process.append("snapshot")
        else:
            incremental_dates = (dates_per_client or {}).get(client_id, [])
            if not incremental_dates:
                print(f"⚠️ No incremental dates for {dataset_name} | {client_id}, skipping.")
                continue

            for d in incremental_dates:
                paths_to_process.append(f"event_date={d}")

        # ------------------------------------------------------------------
        # Process each path
        # ------------------------------------------------------------------
        for p in paths_to_process:
            input_path = f"{raw_base_path}/client_id={client_id}/entity={dataset_name}/{p}/"

            try:
                print(f"📂 {dataset_name} | {client_id} | {p}")
                df = spark.read.parquet(input_path)

                # Flatten & explode
                df = flatten_struct_columns(df)
                if explode_col:
                    explode_col_ = explode_col.replace(".", "_")
                    if explode_col_ in df.columns:
                        df = df.withColumn(explode_col_, explode_outer(col(explode_col_)))

                # Sanitize & align schema
                df = sanitize_column_names(df)
                df = ensure_columns_and_reorder(df, desired_columns, column_types)

                # ------------------------------------------------------------------
                # Source date logic (REAL FIX)
                # ------------------------------------------------------------------
                if p.startswith("event_date="):
                    source_date_val = p.split("=")[1]   # incremental
                else:
                    # snapshot → derive from data
                    if "actiondate" in df.columns:
                        source_date_val = (
                            df.selectExpr("date(actiondate) as d")
                              .agg({"d": "max"})
                              .collect()[0][0]
                        )
                    elif "created" in df.columns:
                        source_date_val = (
                            df.selectExpr("date(created) as d")
                              .agg({"d": "max"})
                              .collect()[0][0]
                        )
                    else:
                        source_date_val = RUN_DATE

                source_date_val = str(source_date_val)
                print(source_date_val)

                # ------------------------------------------------------------------
                # Metadata
                # ------------------------------------------------------------------
                df = (
                    df.withColumn("client_id", lit(client_id))
                      .withColumn("source_date", lit(source_date_val))
                      .withColumn("client_id_col", lit(client_id))
                      .withColumn("source_date_col", lit(source_date_val))
                      .withColumn("processing_timestamp", current_timestamp())
                      .withColumn("processing_date", current_date())
                      .withColumn(
                          "load_type",
                          lit("snapshot" if snapshot_mode else "incremental")
                      )
                )

                # Deduplication
                safe_keys = [k.replace(".", "_") for k in unique_keys]
                df = df.dropDuplicates(safe_keys + ["client_id", "source_date"])
                df = fix_void_columns(df)

                # ------------------------------------------------------------------
                # Write
                # ------------------------------------------------------------------
                write_mode = "overwrite" if snapshot_mode else "append"

                df.write \
                    .mode(write_mode) \
                    .partitionBy("client_id", "source_date") \
                    .parquet(entity_path)

                print(f"✅ Written {df.count()} records")

            except Exception as e:
                print(f"⚠️ Skipped {dataset_name} | {client_id} | {p}: {e}")

# ============================================================================
# RUN
# ============================================================================


# process_dataset(
#     RAW_PATH, CURATED_PATH, CLIENT_IDS,
#     "campaign",
#     unique_keys=["id"],  # deduplicate by campaign id
#     dates_per_client= None,
#     desired_columns=[
#         "object", "id", "title", "created", "isarchived", "ispaused", "wizardstatus", "url",
#         "sender_object", "sender_id", "sender_emailaddress", "sender_fromname", "sender_created",
#         "message_id", "message_ispaused", "message_object", "message_replytoid", "message_subject", "message_type"
#     ],
#     column_types= None
# )

# dates_per_client = get_dates_to_process(
#     curated_path=f"{CURATED_PATH}/entity=activity_open",
#     raw_base_path=RAW_PATH,
#     dataset_name="activity_open",
#     client_ids=CLIENT_IDS,
#     single_date=None
# )

# -------------------- activity_open --------------------
process_dataset(
    RAW_PATH, CURATED_PATH, CLIENT_IDS,
    "activity_open",
    unique_keys=["id", "recipient.id", "campaign.id"],
    dates_per_client=dates_per_client,
    desired_columns = [
        "object", "id", "actiondate", "isduplicate", "recipient_object",
        "recipient_id", "recipient_emailaddress", "recipient_fullname",
        "recipient_created", "recipient_ispaused", "recipient_contactid",
        "recipient_first", "recipient_last", "recipient_fields_link",
        "recipient_fields_status", "recipient_fields_first",
        "recipient_fields_position", "recipient_fields_date_applied",
        "recipient_fields_account", "recipient_fields_phonenumber",
        "recipient_fields_facebookurl", "recipient_fields_instagramid",
        "recipient_fields_linkedinurl", "recipient_fields_twitterid",
        "campaign_object", "campaign_id", "campaign_title", "campaign_wizardstatus",
        "parent_object", "parent_id", "parent_type", "parent_message_object",
        "parent_message_id", "parent_message_type", "parent_message_subject",
        "parent_message_replytoid"
    ],
    column_types = {"recipient_fields_status": StringType(),
                    "recipient_fields_first": StringType(),
                   "parent_message_replytoid": LongType()}
)

dates_per_client = get_dates_to_process(
    curated_path=f"{CURATED_PATH}/entity=activity_reply",
    raw_base_path=RAW_PATH,
    dataset_name="activity_reply",
    client_ids=CLIENT_IDS,
    single_date=None
)

# -------------------- activity_reply --------------------
process_dataset(
    RAW_PATH, CURATED_PATH, CLIENT_IDS,
    "activity_reply",
    unique_keys=["id", "recipient.id", "campaign.id"],
    dates_per_client=dates_per_client,
    desired_columns = [
        "object", "id", "actiondate", "type", "subject", "externalid",
        "externalrawmessageid", "externalconversationid", "rawbody", "body", "plaintextbody",
        "recipient_object", "recipient_id", "recipient_emailaddress", "recipient_fullname",
        "recipient_created", "recipient_ispaused", "recipient_contactid",
        "recipient_first", "recipient_last", "recipient_fields_link",
        "recipient_fields_status",  "recipient_fields_first", "recipient_fields_position", "recipient_fields_date_applied",
        "recipient_fields_account", "recipient_fields_phonenumber", "recipient_fields_facebookurl",
        "recipient_fields_instagramid", "recipient_fields_linkedinurl", "recipient_fields_twitterid",
        "campaign_object", "campaign_id", "campaign_title", "campaign_wizardstatus",
        "parent_object", "parent_id", "parent_type", "parent_message_object", "parent_message_id",
        "parent_message_type", "parent_message_subject", "parent_message_replytoid",
        "from_object", "from_address", "from_fullname", "from_first", "from_last"
    ],
    column_types = {"recipient_fields_status": StringType(), 
                    "recipient_fields_first": StringType(),
                   "parent_message_replytoid": LongType()}
)

dates_per_client = get_dates_to_process(
    curated_path=f"{CURATED_PATH}/entity=activity_sent",
    raw_base_path=RAW_PATH,
    dataset_name="activity_sent",
    client_ids=CLIENT_IDS,
    single_date=None
)

# -------------------- activity_sent --------------------
process_dataset(
    RAW_PATH, CURATED_PATH, CLIENT_IDS,
    "activity_sent",
    unique_keys=["id", "recipient.id", "campaign.id"],
    explode_col="to",
    dates_per_client=dates_per_client,
    desired_columns = [
        "object", "id", "actiondate", "type", "excludebody",
        "to_address", "to_first", "to_fullname", "to_last", "to_object",
        "subject", "externalid", "externalrawmessageid", "externalconversationid", "rawbody", "body", "plaintextbody",
        "recipient_object", "recipient_id", "recipient_emailaddress", "recipient_fullname", "recipient_created", "recipient_ispaused",
        "recipient_first", "recipient_last",
        "recipient_fields_account", "recipient_fields_phonenumber",
        "recipient_fields_facebookurl", "recipient_fields_instagramid",
        "recipient_fields_linkedinurl", "recipient_fields_twitterid",
        "recipient_fields_link", "recipient_fields_position", "recipient_fields_date_applied", "recipient_fields_status",
        "campaign_object", "campaign_id", "campaign_title", "campaign_wizardstatus",
        "message_object", "message_id", "message_type", "message_subject", "message_replytoid",
        "from_object", "from_address", "from_fullname", "from_first", "from_last"
    ],
    column_types={"recipient_fields_status": StringType(),
                 "message_replytoid": DoubleType()}
)

# -------------------- created_leads --------------------
# process_dataset(
#     RAW_PATH, CURATED_PATH, CLIENT_IDS,
#     "created_leads",
#     unique_keys=["id", "recipient.id", "campaign.id"],
#     dates_per_client=dates_per_client,
#     desired_columns = [
#         "object", "id", "created", "openeddate", "laststatuschangedate", "annotation", "status",
#         "recipient_object", "recipient_id", "recipient_emailaddress", "recipient_fullname", "recipient_created", "recipient_ispaused",
#         "recipient_contactid", "recipient_first", "recipient_last",
#         "recipient_fields_link", "recipient_fields_first", "recipient_fields_status",
#         "recipient_fields_position", "recipient_fields_date_applied", "recipient_fields_account",
#         "recipient_fields_phonenumber", "recipient_fields_facebookurl", "recipient_fields_instagramid",
#         "recipient_fields_linkedinurl", "recipient_fields_twitterid",
#         "campaign_object", "campaign_id", "campaign_title", "campaign_wizardstatus",
#         "assignedto_object", "assignedto_id", "assignedto_emailaddress", "assignedto_fullname",
#         "assignedto_first", "assignedto_last"
#     ],
#     column_types={
#         "recipient_fields_status": StringType(),
#         "recipient_fields_first": StringType(),
#         "assignedto_object": StringType(),
#         "assignedto_id": DoubleType(),
#         "assignedto_emailaddress": StringType(),
#         "assignedto_fullname": StringType(),
#         "assignedto_first": StringType(),
#         "assignedto_last": StringType()
#     }
# )

spark.stop()
print("🎉 All datasets processed successfully!")


Loaded clients: ['client_4', 'client_5', 'client_6']
📂 activity_open | client_4 | snapshot
⚠️ Adding missing column: recipient_fields_status
⚠️ Adding missing column: recipient_fields_first
2026-01-02


✅ Written 100 records
📂 activity_open | client_5 | snapshot
⚠️ Adding missing column: recipient_fields_status
⚠️ Adding missing column: recipient_fields_first
2026-01-02


✅ Written 100 records
📂 activity_open | client_6 | snapshot
⚠️ Adding missing column: recipient_fields_status
⚠️ Adding missing column: recipient_fields_first
2026-01-02


✅ Written 100 records
Loaded last_dates from curated:
  client_12: 2026-01-04
  client_2: 2025-12-25
  client_8: 2026-01-05
  client_9: 2026-01-05
  client_10: 2026-01-04
  client_11: 2026-01-04
  client_7: 2026-01-05
  client_3: 2025-12-25
  client_1: 2025-12-25
📂 activity_reply | client_4 | snapshot
⚠️ Adding missing column: recipient_fields_status
⚠️ Adding missing column: recipient_fields_first
2026-01-02


✅ Written 25 records
📂 activity_reply | client_5 | snapshot
⚠️ Adding missing column: recipient_fields_status
⚠️ Adding missing column: recipient_fields_first
2026-01-02


✅ Written 25 records
📂 activity_reply | client_6 | snapshot
⚠️ Adding missing column: recipient_fields_status
⚠️ Adding missing column: recipient_fields_first
2026-01-02


✅ Written 25 records
Loaded last_dates from curated:
  client_10: 2026-01-04
  client_3: 2025-12-25
  client_8: 2026-01-05
  client_2: 2025-12-25
  client_7: 2026-01-05
  client_1: 2025-12-25
  client_12: 2026-01-04
  client_9: 2026-01-05
  client_11: 2026-01-04
📂 activity_sent | client_4 | snapshot
⚠️ Adding missing column: recipient_fields_status
2026-01-02


✅ Written 25 records
📂 activity_sent | client_5 | snapshot
⚠️ Adding missing column: recipient_fields_status
2026-01-02


✅ Written 25 records
📂 activity_sent | client_6 | snapshot
⚠️ Adding missing column: recipient_fields_status
2026-01-02


✅ Written 25 records
🎉 All datasets processed successfully!
